##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using TFRecords and `tf.Example`

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/tf-records"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/tf-records.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/tf-records.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

To read data efficiently it can be helpful to serialize your data and store it in a set of files (100-200MB each) that can each be read linearly. This is especially true if the data is being streamed over a network. This can also be useful for caching any data-preprocessing.

The TFRecord format is a simple format for storing a sequence of binary records.

[Protocol buffers](https://developers.google.com/protocol-buffers/) are a cross-platform, cross-language library for efficient serialization of structured data.

Protocol messages are defined by `.proto` files, these are often the easiest way to understand a message type. 

The `tf.Example` message (or protobuf) is a flexible message type that represents a `{"string": value}` mapping. It is designed for use with TensorFlow, and is used throughout the higher-level APIs such as [TFX](https://www.tensorflow.org/tfx/).


This notebook will demonstrate how to create, parse, and use the `tf.Example` message, and then store, read, and write `tf.Example` messages to `.tfrecord` files.

Note: While useful, using these structures is ultimately optional. There is no need to convert existing code to use TFRecords, unless you are using [`tf.data`](https://www.tensorflow.org/guide/datasets) and reading data is still the bottleneck to training. See [Data Input Pipeline Performance](https://www.tensorflow.org/guide/performance/datasets) for dataset performance tips.

## Setup

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import IPython.display as display

## `tf.Example`

### Data types for `tf.Example`

Fundamentally a `tf.Example` is a `{"string": tf.train.Feature}` mapping.

The `tf.train.Feature` message type can accept one of the following three types (See the [`.proto` file]((https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto) for reference). Most other generic types can be coerced into one of these.

1. `tf.train.BytesList` (the following types can be coerced)

  - `string`
  - `byte`

1. `tf.train.FloatList` (the following types can be coerced)

  - `float` (`float32`)
  - `double` (`float64`)

1. `tf.train.Int64List` (the following types can be coerced)

  - `bool`
  - `enum`
  - `int32`
  - `uint32`
  - `int64`
  - `uint64`

In order to convert a standard TensorFlow type to a `tf.Example`-compatible `tf.train.Feature`, we can use the following shortcut functions:

Each function takes a scalar input value and returns a `tf.train.Feature` containing one of the 3 `list` types above.

In [0]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

Note: To stay simple, this example only uses scalar inputs. The simplest way to handle non-scalar features is to use `tf.serialize_tensor` to convert tensors to binary-strings. Strings are scalars in tensorflow. Use `tf.parse_tensor` to convert the binary-string back to a tensor.

Below are some examples of how these functions work. Note the varying input types and the standardizes output types. If the input type for a function does not match one of the coercible types stated above, the function will raise an exception (e.g. `_int64_feature(1.0)` will error out, since `1.0` is a float, so should be used with the `_float_feature` function instead).

In [0]:
print(_bytes_feature('test_string'))
print(_bytes_feature(bytes('test_bytes')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(1))

All proto messages can be serialized to a binary-string using the `.SerializeToString` method.

In [0]:
feature = _float_feature(np.exp(1))

feature.SerializeToString()

### Creating a `tf.Example` message

Suppose you want to create a `tf.Example` message from existing data. In practice, the dataset may come from anywhere, but the procedure of creating the `tf.Example` message from a single observation will be the same. 

1. Within each observation, each value needs to be converted to a `tf.train.Feature` containing one of the 3 compatible types, using one of the functions above. 

1. We create a map (dictionary) from the feature name string to the encoded feature value produced in #1.

1. The map produced in #2 is converted to a [`Features` message](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto#L85).

In this notebook, we will create a dataset using NumPy. 

This dataset will have 4 features.
- a boolean feature, `False` or `True` with equal probability
- a random bytes feature, uniform across the entire support
- an integer feature uniformly randomly chosen from `[-10000, 10000)`
- a float feature from a standard normal distribution

Consider a sample consisting of 10,000 independently and identically distributed observations from each of the above distributions.

In [0]:
# the number of observations in the dataset
n_observations = int(1e4)

# boolean feature, encoded as False or True
feature0 = np.random.choice([False, True], n_observations)

# bytes feature
feature1 = np.random.bytes(n_observations)

# integer feature, random between -10000 and 10000
feature2 = np.random.randint(-10000, 10000, n_observations)

# float feature, from a standard normal distribution
feature3 = np.random.randn(n_observations)

Each of these features can be coerced into a `tf.Example`-compatible type using one of `_bytes_feature`, `_float_feature`, `_int64_feature`. We can then create a `tf.Example` message from these encoded features.

In [0]:
def create_example(features):
  """
  Creates a tf.Example message ready to be written to a file.
  
  Inputs:
    - features: a 4-list of the values in the observation
  """
  
  # Create a dictionary mapping the feature name to the tf.Example-compatible
  # data type.
  
  feature = {
      'feature0': _int64_feature(features[0]),
      'feature1': _bytes_feature(features[1]),
      'feature2': _int64_feature(features[2]),
      'feature3': _float_feature(features[3]),
  }
  
  # Create a Features message using tf.train.Example.
  
  return tf.train.Example(features=tf.train.Features(feature=feature))

For example, suppose we have a single observation from the dataset, `[False, bytes('example'), -1234, 0.9876]`. We can create and print the `tf.Example` message for this observation using `create_message()`. Each single observation will be written as a `Features` message as per the above. Note that the `tf.Example` [message](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/example.proto#L88) is just a wrapper around the `Features` message.

In [0]:
# This is an example observation from the dataset.

example_observation = [False, bytes('example'), -1234, 0.9876]

print(create_example(example_observation))

Or the serialized form:

In [0]:
create_example(example_observation).SerializeToString()

## TFRecord files using tf.python_io

The `tf.python_io` module contains python functions for reading and writing TFRecord files. 

### Writing a TFRecord file

We now write the 10,000 observations to the file `test.tfrecords`. Each observation is converted to a `tf.Example` message, then written to file. We can then verify that the file `test.tfrecords` has been created.

In [0]:
# Write the tf.Example observations to test.tfrecords.

with tf.python_io.TFRecordWriter('test.tfrecords') as writer:
  for i in range(n_observations):
    example = create_example([feature0[i], feature1[i], feature2[i], feature3[i]])
    writer.write(example.SerializeToString())

In [0]:
!ls

### Reading a TFRecord File

Suppose we now want to read this data back, to be input as data into a model.

The following example imports the data as is, as a `tf.Example` message. This can be useful to verify that a the file contains the data that we expect. This can also be useful if the input data is stored as TFRecords but you would prefer to input NumPy data (or some other input data type), for example [here](https://www.tensorflow.org/guide/datasets#consuming_numpy_arrays), since this example allows us to read the values themselves.

We iterate through the TFRecords in the infile, extract the `tf.Example` message, and can read/store the values within.

In [0]:
record_iterator = tf.python_io.tf_record_iterator(path='test.tfrecords')

for string_record in record_iterator:
  example = tf.train.Example()
  example.ParseFromString(string_record)
  
  print(example)
  
  # Exit after 1 iteration as this is purely demonstrative.
  break

The features of the `example` object (created above of type `tf.Example`) can be accessed using its getters (similarly to any protocol buffer message). `example.features` returns a `repeated feature` message, then getting the `feature` message returns a map of feature name to feature value (stored in Python as a dictionary).

In [0]:
print(dict(example.features.feature))

From this dictionary, you can get any given value as with a dictionary.

In [0]:
print(example.features.feature['feature3'])

Now, we can access the value using the getters again.

In [0]:
print(example.features.feature['feature3'].float_list.value)

## TFRecord files using `tf.data`

The `tf.data` module also provides tools for reading and writing TFRecord files. The writer, `tf.data.experimental.TFRecordWriter`, is not covered by this tutorial.

### Reading a TFRecord File

We can also read the TFRecord file using the `tf.data.TFRecordDataset` class. 

More information on consuming TFRecord files using `tf.data` can be found [here](https://www.tensorflow.org/guide/datasets#consuming_tfrecord_data). 

Using this dataset structure can be useful for standardizing input data and optimizing performance. It is also easier and quicker to use this object.

In [0]:
filenames = ['test.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

At this point the dataset contains serialized `tf.train.Example` messages. When iterated over it returns these as scalar string tensors. 

Use the `.take` method to only show the first 10 records.

Note: iterating over a `tf.data.Dataset` only works with eager execution enabled.

In [0]:
for raw_record in raw_dataset.take(10):
  print(repr(raw_record))

These tensors can be parsed using the function below.

Note: The `feature_description` is necessary here because datasets use graph-execution, and need this description to build their shape and type signature.

In [0]:
# Create a description of the features.  
feature_description = {
    'feature0': tf.FixedLenFeature([], tf.int64, default_value=0),
    'feature1': tf.FixedLenFeature([], tf.string, default_value=''),
    'feature2': tf.FixedLenFeature([], tf.int64, default_value=0),
    'feature3': tf.FixedLenFeature([], tf.float32, default_value=0.0),
}

def _parse_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.parse_single_example(example_proto, feature_description)

Or use `tf.parse example` to parse a whole batch at once.

Apply this finction to each item in the dataset using the `tf.data.Dataset.map` method:

In [0]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset 

Now, we can use eager execution to display the observations in the dataset. Note that there are 10,000 observations in this dataset, but we only display the first 10. The data is displayed as a dictionary of features. Each item is a `tf.Tensor`, and the `numpy` element of this tensor displays the value of the feature.

In [0]:
for parsed_record in parsed_dataset.take(10):
  print(repr(raw_record))

## Walkthrough: Reading/Writing Image Data

This is an example of how to read and write image data using TFRecords. The purpose of this is to show how, end to end, input data (in this case an image) and write the data as a TFRecord file, then read the file back and display the image.

This can be useful if, for example, you want to use several models on the same input dataset. Instead of storing the image data raw, it can be preprocessed into the TFRecords format, and that can be used in all further processing and modelling. 

First, let's download [this](https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg) adorable image of a cat in the snow, and [this](https://upload.wikimedia.org/wikipedia/commons/f/fe/New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg) awesome picture of the Williamsburg Bridge, NYC under construction.

### Fetch the images

In [0]:
cat_in_snow  = tf.keras.utils.get_file('320px-Felis_catus-cat_on_snow.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Felis_catus-cat_on_snow.jpg/320px-Felis_catus-cat_on_snow.jpg')
williamsburg_bridge = tf.keras.utils.get_file('194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg','https://upload.wikimedia.org/wikipedia/commons/thumb/f/fe/New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg/194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg')

In [0]:
display.Image(filename=cat_in_snow)

In [0]:
display.Image(filename=williamsburg_bridge)

### Write the TFRecord file

As we did earlier, we can now encode the features as types compatible with `tf.Example`. In this case, we will not only store the raw image string as a feature, but we will store the height, width, depth, and an arbitrary `label` feature, which is used when we write the file to distinguish between the cat image and the bridge image. We will use `0` for the cat image, and `1` for the bridge image. 

In [0]:
image_labels = {
    cat_in_snow : 0,
    williamsburg_bridge : 1,
}

In [0]:
# This is an example, just using the cat image.
image_string = open(cat_in_snow, 'rb').read()

label = image_labels[cat_in_snow]

# Create a dictionary with features that may be relevant.
def image_example(image_string, label):
  image_shape = tf.image.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

for line in str(image_example(image_string, label)).split('\n')[:15]:
  print(line)
print('...')

We see that all of the features are now stores in the `tf.Example` message. Now, we functionalize the code above and write the example messages to a file, `images.tfrecords`.

In [0]:
# Write the raw image files to images.tfrecords.
# First, process the two images into tf.Example messages.
# Then, write to a .tfrecords file.

with tf.python_io.TFRecordWriter('images.tfrecords') as writer:
  for filename, label in image_labels.items():
    image_string = open(filename, 'rb').read()
    tf_example = image_example(image_string, label)
    writer.write(tf_example.SerializeToString())

In [0]:
!ls

### Read the TFRecord file

We now have the file `images.tfrecords`. We can now iterate over the records in the file to read back what we wrote. Since, for our use case we will just reproduce the image, the only feature we need is the raw image string. We can extract that using the getters described above, namely `example.features.feature['image_raw'].bytes_list.value[0]`. We also use the labels to determine which record is the cat as opposed to the bridge.

In [0]:
raw_image_dataset = tf.data.TFRecordDataset('images.tfrecords')

# Create a dictionary describing the features.  
image_feature_description = {
    'height': tf.FixedLenFeature([], tf.int64),
    'width': tf.FixedLenFeature([], tf.int64),
    'depth': tf.FixedLenFeature([], tf.int64),
    'label': tf.FixedLenFeature([], tf.int64),
    'image_raw': tf.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

Now we can recover the images from the TFRecord file:

In [0]:
for image_features in parsed_image_dataset:
  image_raw = image_features['image_raw'].numpy()
  display.display(display.Image(data=image_raw))